# Apply 

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(12,6)})
plt.style.use(['seaborn-whitegrid'])

import warnings
warnings.simplefilter('ignore')

import utils

In [ ]:
filenames = ['data/measured_real_power.csv',
             'data/measured_reactive_power.csv']

df = utils.read_dir(filenames)
df.shape

What is the `real` and `reactive` timeseries for `triplex_meter_5`?

In [ ]:
tm5 = df.query("variable=='triplex_meter_5'")
tm5.head()

Pivot on **attribute**

In [ ]:
tm5 = tm5.pivot(index='timestamp', columns='attribute', values='value')

In [ ]:
_ = tm5.plot(alpha=0.75)

## The `apply` function 

In [ ]:
tm5.head()

In [ ]:
def func(x):
    return "just a string {} {}".format(x.shape, type(x))
    
tm5.apply(func, axis=0)  # flip the axis

In [ ]:
tm5.apply(lambda x: "just a string {}".format(", ".join(x.index.values)), axis=1).head()

In [ ]:
tmp = tm5.copy()
tmp['func'] = tmp.apply(func, axis=1)
tmp.head()

How about the `row` magnitude of `real` and `reactive`?

In [ ]:
def row_mag(x):
    return abs(complex(x['measured_real_power'], x['measured_reactive_power']))

tm5.apply(row_mag, axis=1).head()

In [ ]:
tm5.apply(row_mag, axis=1).resample('60min').mean().plot()

Two other methods

In [ ]:
import math

def row_mag_basic(x):
    return math.sqrt(x['measured_real_power']*x['measured_real_power'] + 
                     x['measured_reactive_power']*x['measured_reactive_power'])

def compute_mag(df):
    total =  (df['measured_real_power']*df['measured_real_power'] + 
              df['measured_reactive_power']*df['measured_reactive_power'])
    
    return total.apply(math.sqrt)

We cannot do:
    
```python
        math.sqrt(total)
```
but we can `apply` it.

In [ ]:
tm5.apply(row_mag, axis=1).resample('60min').sum().plot()
compute_mag(tm5).resample('60min').sum().plot(kind='area', alpha=0.1)

In [ ]:
%timeit tm5.apply(row_mag, axis=1)

In [ ]:
%timeit tm5.apply(row_mag_basic, axis=1)

In [ ]:
%timeit compute_mag(tm5)

Pandas stores data in `column-wise` numpy arrays.